
    *****************************************************
    Programmer:    Adeyemi Adedoyin Simeon
    Version:       1.0
    Contact:       adeyemi.sa1@gmail.com, +2348064555381
    Github:        https://github.com/SimeonDee
    Location:      Offa, Kwara State, Nigeria
    Date:          12th April, 2020 12:00pm
    *****************************************************
    
# A Python API to connect Python to MYSQL Database and execute SQL statements

### The API will automatically install the mysql-connector-python, an api for connecting python to mysql, if not already installed.

You can view this source code and download the api from my github page.
Follow the link 
## https://github.com/SimeonDee/python-mysql-doyin-connect-api'

In [46]:
import mysql.connector as mysql_api
from mysql.connector import Error
from requests import ConnectionError
import pandas as pd
from os import path

class mysql_connect_doyin_api:
    """
    A Python API to connect Python to MYSQL Database and execute SQL statements. It automatically installs 'mysql-connector-python' if not already installed
    """
    def __init__(self, conn_args={}):
        """
        conn_args: A dictionary containing key:value pair of all MySQL connection parameters
        
        Example:
        --------
            conn_args = {'host':'localhost', 'database':'python_db', 'user':'root', 'password':'', 'autocommit':True, 'raise_on_warnings': True}
        
        For detailed list and explanation of all MySQL connection parameters, 
        visit https://pynative.com/python-mysql-database-connection/
        """
        
        self.conn_args = conn_args
        self.conn_obj = None
        
        if path.exists('C:\ProgramData\Anaconda3\Lib\site-packages\mysql\connector') == False:    
            try:
                # Installs mysql connector for python if not already installed
                print('Installing mysql-connector-python ...\n')
                !python -m pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org --trusted-host pypi.python.org mysql-connector-python
                print('Now Intsalled')

            except ConnectionError as e:
                print('Internet Required to install mysql-connector-python, if not already installed. \nError:{}'.format(e))
                print('You can also install directly with "pip install mysql-connector-python"')


        
    def connect_to_mysql(self):
        """
        Establishes connection to MySQL database using the connection arguments used at instantiation and returns a connection object
        """
        try:
            if type(self.conn_args) == dict and len(self.conn_args) > 0:
                conn = mysql_api.connect(**self.conn_args)

                if conn.is_connected():
                    db_info = conn.get_server_info()
                    print('Connection established to MySQL Database Server ', db_info)
                    self.conn_obj = conn
                else:
                    self.conn_obj = None

            else:
                self.conn_obj = None

        except Error as e:
            print('Error connecting to MySQL database: {}'.format(e))
            
    
    def disconnect_mysql(self):
        try:
            if type(self.conn_obj) == mysql_api.connection.MySQLConnection:
                if self.conn_obj.is_connected():
                    self.conn_obj.close()
                    print('Connection to database closed')

                else:
                    print('Connection to database is already closed')

            else:
                print('Argument has to be an active mysql connection object.')

        except Error as e:
            print('Error connecting to MySQL database: {}'.format(e))


    def mysql_run_sql(self, sql="""""", data=None):
        """
        Runs 'sql' with optional 'data' parameter values on 'con_obj' connection object of mysql database

        data: Must be a tuple or list of tuples e.g. ('Wale', 2) or [('Wale', 2), ('Kola', 3)]
        """
        try:    
            if self.conn_obj != None:
                cursor = self.conn_obj.cursor()
                if data != None and (type(data) == tuple or type(data) == list):
                    if type(data) == list and len(data) > 1:
                        result = cursor.executemany(sql, data)
                    else:
                        result = cursor.execute(sql, data)

                else:
                    result = cursor.execute(sql)

                self.conn_obj.commit()

                print('Excuted Successfully.')

            else:
                print('"con_obj" argument cannot be None')

        except Error as e:
            print('Error executing sql: {}'.format(e))

            
    def mysql_run_fetch(self, sql="""""", data=None):
        """
        Runs 'sql' with optional 'data' parameter values on an active connection object of mysql database

        data: Must be a tuple or list of tuples e.g. ('Wale', 2) or [('Wale', 2), ('Kola', 3)]
        """
        try:    
            if self.conn_obj != None:
                cursor = self.conn_obj.cursor()
                if data != None and (type(data) == tuple or type(data) == list):
                    cursor.execute(sql, data)

                else:
                    cursor.execute(sql)

                # cursor.fetchone() to fetch one record
                # cursor.fetchmany(SIZE) to fetch SIZE records .i.e. limited fetch
                result = cursor.fetchall()


                print('Excuted Successfully.')

                return result

            else:
                print('"conn_obj" argument cannot be None')
                return None

        except Error as e:
            print('Error executing sql: {}'.format(e))

# ================ SAMPLE USAGE =================

# Usage

In [48]:
# Instantiating the API
conn_args = {'host':'localhost', 'database':'python_db', 
            'user':'root', 'password':'', 'autocommit':True, 
            'raise_on_warnings': True}


mysql_conn_api = mysql_connect_doyin_api(conn_args=con_args)

In [49]:
# Connecting to Database
mysql_conn_api.connect_to_mysql()

Connection established to MySQL Database Server  5.5.5-10.1.38-MariaDB


In [50]:
# Running a DDL SQL command
sql = """CREATE TABLE IF NOT EXISTS Person(id int NOT NULL AUTO_INCREMENT,
            Fullname VARCHAR(70) NOT NULL,
            Gender VARCHAR(7),
            PRIMARY KEY(id)); """

# mysql_conn_api.mysql_run_sql(sql=sql)

In [51]:
# Inserting record(s) into database
sql = """INSERT IGNORE INTO Person(Fullname, Gender) VALUES(%s, %s); """
input_data = [('Adeyemi Adedoyin', 'Male'), ('Kolawole Sandra', 'Female'), ('David Joe', 'Male')]

mysql_conn_api.mysql_run_sql(sql=sql, data=input_data)

Excuted Successfully.


In [52]:
# Fetching data from database table
sql = """SELECT * FROM Person; """

result = mysql_conn_api.mysql_run_fetch(sql=sql)

if result != None:
    for row in result:
        print(row)
        

# Converting result into dataframe
data = {'SNo':[x[0] for x in result], 'Fullname':[x[1] for x in result], 'Gender':[x[2] for x in result]}
df = pd.DataFrame(data=data)
df

Excuted Successfully.
(1, 'Samson Adeola', 'Male')
(2, 'Badiru Mary', 'Female')
(3, 'Samson Adeola', 'Male')
(4, 'Badiru Mary', 'Female')
(5, 'Adewale', 'Male')
(6, 'Adisa Michael', 'Male')
(7, 'Omisoore Aminat', 'Female')
(8, 'Sanjo Dauda', 'Male')
(9, 'Adisa Michael', 'Male')
(10, 'Omisoore Aminat', 'Female')
(11, 'Sanjo Dauda', 'Male')
(12, 'Adeyemi Adedoyin', 'Male')
(13, 'Kolawole Sandra', 'Female')
(14, 'David Joe', 'Male')


,SNo,Fullname,Gender
0,1,Samson Adeola,Male
1,2,Badiru Mary,Female
2,3,Samson Adeola,Male
3,4,Badiru Mary,Female
4,5,Adewale,Male
5,6,Adisa Michael,Male
6,7,Omisoore Aminat,Female
7,8,Sanjo Dauda,Male
8,9,Adisa Michael,Male
9,10,Omisoore Aminat,Female


In [53]:
mysql_conn_api.disconnect_mysql()

Connection to database closed


# ================ THE END =================